In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
medium_data = pd.read_csv('/kaggle/input/medium-articles-dataset/medium_data.csv')
medium_data.head(10)

,id,url,title,subtitle,image,claps,responses,reading_time,publication,date
0,1,https://towardsdatascience.com/a-beginners-gui...,A Beginner’s Guide to Word Embedding with Gens...,NaN,1.png,850,8,8,Towards Data Science,2019-05-30
1,2,https://towardsdatascience.com/hands-on-graph-...,Hands-on Graph Neural Networks with PyTorch & ...,NaN,2.png,1100,11,9,Towards Data Science,2019-05-30
2,3,https://towardsdatascience.com/how-to-use-ggpl...,How to Use ggplot2 in Python,A Grammar of Graphics for Python,3.png,767,1,5,Towards Data Science,2019-05-30
3,4,https://towardsdatascience.com/databricks-how-...,Databricks: How to Save Files in CSV on Your L...,When I work on Python projects dealing…,4.jpeg,354,0,4,Towards Data Science,2019-05-30
4,5,https://towardsdatascience.com/a-step-by-step-...,A Step-by-Step Implementation of Gradient Desc...,One example of building neural…,5.jpeg,211,3,4,Towards Data Science,2019-05-30
5,6,https://towardsdatascience.com/an-easy-introdu...,An Easy Introduction to SQL for Data Scientists,NaN,6.jpeg,563,1,8,Towards Data Science,2019-05-30
6,7,https://towardsdatascience.com/hypothesis-test...,Hypothesis testing visualized,Literally seeing how stat tests work,7.gif,276,1,12,Towards Data Science,2019-05-30
7,8,https://towardsdatascience.com/introduction-to...,Introduction to Latent Matrix Factorization Re...,NaN,8.png,126,2,5,Towards Data Science,2019-05-30
8,9,https://towardsdatascience.com/which-2020-cand...,Which 2020 Candidate is the Best at Twitter?,A Data Analysis of the 2020 Democratic Candida...,9.png,188,3,8,Towards Data Science,2019-05-30
9,10,https://towardsdatascience.com/what-if-ai-mode...,What if AI model understanding were easy?,See analytics-for-AI in action in this What-I...,10.jpg,493,2,8,Towards Data Science,2019-05-30


In [4]:
medium_data.shape[0:2:1]

(6508, 10)

In [5]:
medium_data['title']

0       A Beginner’s Guide to Word Embedding with Gens...
1       Hands-on Graph Neural Networks with PyTorch & ...
2                            How to Use ggplot2 in Python
3       Databricks: How to Save Files in CSV on Your L...
4       A Step-by-Step Implementation of Gradient Desc...
                              ...                        
6503    “We” vs “I” — How Should You Talk About Yourse...
6504                     How Donald Trump Markets Himself
6505        Content and Marketing Beyond Mass Consumption
6506    5 Questions All Copywriters Should Ask Clients...
6507               How To Write a Good Business Blog Post
Name: title, Length: 6508, dtype: object

In [6]:
medium_data['title'] = medium_data['title'].apply(lambda x: x.replace(u'\xa0',u' '))

medium_data['title'] = medium_data['title'].apply(lambda x: x.replace('\u200a',' '))

In [7]:
medium_data['title']

0       A Beginner’s Guide to Word Embedding with Gens...
1       Hands-on Graph Neural Networks with PyTorch & ...
2                            How to Use ggplot2 in Python
3       Databricks: How to Save Files in CSV on Your L...
4       A Step-by-Step Implementation of Gradient Desc...
                              ...                        
6503    “We” vs “I” — How Should You Talk About Yourse...
6504                     How Donald Trump Markets Himself
6505        Content and Marketing Beyond Mass Consumption
6506    5 Questions All Copywriters Should Ask Clients...
6507               How To Write a Good Business Blog Post
Name: title, Length: 6508, dtype: object

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer

import pickle

tokenizer = Tokenizer(oov_token='<oov>')

tokenizer.fit_on_texts(medium_data['title'])

total_words = len(tokenizer.word_index) + 1

pickle.dump(tokenizer,open('token.pkl','wb'))

total_words


8238

In [9]:
input_sequences = []

for line in medium_data['title']:
    
    token_list = tokenizer.texts_to_sequences([line])[0]
    
    for i in range(1, len(token_list)):
        
        n_gram_sequence = token_list[:i+1]
        
        input_sequences.append(n_gram_sequence)

len(input_sequences)

48461

In [10]:

from tensorflow.keras.preprocessing.sequence import pad_sequences

max_sequence_len = max([len(x) for x in input_sequences])

input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

input_sequences[1]



array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   5, 676,
        68], dtype=int32)

In [11]:
import tensorflow as tf

X, labels = input_sequences[:,:-1],input_sequences[:,-1]

Y = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [12]:
from tensorflow.keras.layers import LSTM,Embedding,Dense

from tensorflow.keras.models import Sequential

model = Sequential()

model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))

model.add(LSTM(150,return_sequences=True))

model.add(LSTM(150,return_sequences=True))

model.add(LSTM(150,return_sequences=True))

model.add(LSTM(150))

model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X, Y, epochs=150, verbose=1)

print(model)


Epoch 1/150
1515/1515 [==============================] - 39s 19ms/step - loss: 7.0507 - accuracy: 0.0537
Epoch 2/150
1515/1515 [==============================] - 29s 19ms/step - loss: 6.6739 - accuracy: 0.0947
Epoch 3/150
1515/1515 [==============================] - 29s 19ms/step - loss: 6.5750 - accuracy: 0.1043
Epoch 4/150
1515/1515 [==============================] - 30s 20ms/step - loss: 6.5124 - accuracy: 0.1120
Epoch 5/150
1515/1515 [==============================] - 29s 19ms/step - loss: 6.4507 - accuracy: 0.1184
Epoch 6/150
1515/1515 [==============================] - 29s 19ms/step - loss: 6.3891 - accuracy: 0.1228
Epoch 7/150
1515/1515 [==============================] - 29s 19ms/step - loss: 6.3474 - accuracy: 0.1286
Epoch 8/150
1515/1515 [==============================] - 30s 20ms/step - loss: 6.2987 - accuracy: 0.1310
Epoch 9/150
1515/1515 [==============================] - 30s 20ms/step - loss: 6.3057 - accuracy: 0.1363
Epoch 10/150
1515/1515 [==============================]

In [18]:
model.save('Next_word_pred_Median_data.h5.h5')